In [1]:
%pylab inline
import pandas as pd
import networkx as nx

import dash
import dash_cytoscape as cyto
import dash_html_components as html
from jupyterlab_dash import AppViewer
viewer = AppViewer()

from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import json

import dash_reusable_components as drc

app = dash.Dash(__name__)

Populating the interactive namespace from numpy and matplotlib


ModuleNotFoundError: No module named 'dash_reusable_components'

In [2]:
edges=pd.read_csv('edges_of_graph.csv')
edges=edges.drop('Unnamed: 0',axis=1)

names=pd.read_csv('names_of_nodes.csv')
names=names.drop('Unnamed: 0',axis=1)

In [3]:
s=[]
for i in list(edges.source):
    s.append(int(i))
t=[]
for j in list(edges.target):
    t.append(int(j))
#n=list(names['0'])

In [4]:
el=[]
for i in range(len(s)):
    source=s[i]
    target =t[i]
    lab_s=names['0'][source]
    lab_t=names['0'][target]
    
    el.append({'data': {'id': source, 'label': lab_s} })
    el.append({'data': {'id': target, 'label': lab_t} })
    el.append({'data': {'source': source, 'target': target}})

app = dash.Dash(__name__)
app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape',
        elements=el,
        layout={'name': 'cose'}
    )
])
viewer.show(app)

In [5]:
app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown-update-layout',
        value='cose',
        clearable=False,
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['cose', 'grid', 'random', 'circle',  'concentric']
        ]
    ),
    cyto.Cytoscape(
        id='cytoscape-update-layout',
        layout={'name': 'cose'},
        style={'width': '100%', 'height': '450px'},
        elements=el
    ),
    dcc.Markdown(id='cytoscape-selectedNodeData-markdown')
])


@app.callback(Output('cytoscape-update-layout', 'layout'),
              [Input('dropdown-update-layout', 'value')])
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

@app.callback(Output('cytoscape-selectedNodeData-markdown', 'children'),
                  [Input('cytoscape-update-layout', 'selectedNodeData')])
def displaySelectedNodeData(data_list):
    if not data_list:
        return

    cities_list = [data['label'] for data in data_list]
    return "You selected the following editor(s): " + "\n* ".join(cities_list)

In [6]:
viewer.show(app)